In [0]:
import pandas as pd
import numpy as np

# Mid-Course Project

Hi There, and thanks for your help. If you're reading this you've been selected to help on a secret initiative.

You will be helping us analyze a portion of data from a company we want to acquire, which could greatly improve the fortunes of Maven Mega Mart.

We'll be working with `project_transactions.csv` and briefly take a look at `product.csv`.

First, read in the transactions data and explore it.

* Take a look at the raw data, the datatypes, and cast `DAY`, `QUANTITY`, `STORE_ID`, and `WEEK_NO` columns to the smallest appropriate datatype. Check the memory reduction by doing so.
* Is there any missing data?
* How many unique households and products are there in the data? The fields household_key and Product_ID will help here.

In [0]:
transactions = pd.read_csv("../project_data/project_transactions.csv",
                          dtype={"DAY": "Int16",
                                 "QUANTITY": "Int32",
                                 "STORE_ID": "Int32",
                                 "WEEK_NO": "Int8",}
                          )

In [0]:
transactions.head()

In [0]:
# Reduced memory usage by ~35MB after converting to correct dtypes

transactions.info(memory_usage="deep")

In [0]:
# Use to identify which values can be downcast

transactions.describe().round()

In [0]:
# Check missing value counts

transactions.isna().sum()

In [0]:
# Calculate unique households in dataset with nunique (describe could also be used)

transactions["household_key"].nunique()

In [0]:
# Calculate unique product_ids in dataset with nunique

transactions["PRODUCT_ID"].nunique()

## Column Creation

Create two columns:

* A column that captures the `total_discount` by row (sum of `RETAIL_DISC`, `COUPON_DISC`)
* The percentage disount (`total_discount` / `SALES_VALUE`). Make sure this is positive (try `.abs()`).
* If the percentage discount is greater than 1, set it equal to 1. If it is less than one, set it to 0. 
* Drop the individual discount columns (`RETAIL_DISC`, `COUPON_DISC`, `COUPON_MATCH_DISC`).

Feel free to overwrite the existing transaction DataFrame after making the modifications above.

In [0]:
# create a discount sum column and a percentage discount column
transactions = (
    transactions
    .assign(total_discount = transactions["RETAIL_DISC"] + transactions["COUPON_DISC"],
            percentage_discount = (lambda x: (x["total_discount"] / x["SALES_VALUE"]).abs()))
    .drop(["RETAIL_DISC", "COUPON_DISC", "COUPON_MATCH_DISC"], axis=1))


# Use where to cap values above 1 at 1 and below 0 at 0. Other methods could be used here as well.
transactions["percentage_discount"] = (transactions["percentage_discount"]
                                       .where(transactions["percentage_discount"] < 1, 1.0)
                                       .where(transactions["percentage_discount"] > 0, 0)
                                      )

transactions.head()

## Overall Statistics

Calculate:

* The total sales (sum of `SALES_VALUE`), 
* Total discount (sum of `total_discount`)
* Overall percentage discount (sum of total_discount / sum of sales value)
* Total quantity sold (sum of `QUANTITY`).
* Max quantity sold in a single row. Inspect the row as well. Does this have a high discount percentage?
* Total sales value per basket (sum of sales value / nunique basket_id).
* Total sales value per household (sum of sales value / nunique household_key). 
* What was the highest quantiy sold for a single item in a single row?

In [0]:
# Total Sales Value

transactions['SALES_VALUE'].sum()

In [0]:
# Total Discount

transactions['total_discount'].sum()

In [0]:
# Overall Percent Discount

transactions['total_discount'].sum() / transactions['SALES_VALUE'].sum()

In [0]:
# Average of pct_discount column

transactions["percentage_discount"].mean()

In [0]:
# Total Quantity Sold

transactions['QUANTITY'].sum()

In [0]:
# Max quantity in single row

transactions["QUANTITY"].max()

In [0]:
# Use to grab row with max value - discount rate is lower than average

transactions.loc[transactions["QUANTITY"].argmax()]

In [0]:
# Sales value per transaction/basket

transactions['SALES_VALUE'].sum() / transactions['BASKET_ID'].nunique()

In [0]:
# Sales value per household

transactions['SALES_VALUE'].sum() / transactions['household_key'].nunique()

## Household Analysis

* Plot the distribution of total sales value purchased at the household level. 
* What were the top 10 households by quantity purchased?
* What were the top 10 households by sales value?
* Plot the total sales value for our top 10 households by value, ordered from highest to lowest.


In [0]:
# plot distribution of households by total sales value
# First groupby household and calculate sum of sales
# then plot with a histogram

(transactions
 .groupby("household_key")
 .agg({'SALES_VALUE': 'sum'})
 .plot.hist())

In [0]:
# store top 10 households by total value and quantity
# groupby household_key, calculate sum of relevant columns by household
# sort both by relevant metric in descending order, and grab top 10 rows

top10_value = (transactions
 .groupby("household_key")
 .agg({'SALES_VALUE': 'sum'})
 .sort_values("SALES_VALUE", ascending=False)
 .iloc[:10])

top10_quant = (transactions
 .groupby("household_key")
 .agg({"QUANTITY": "sum"})
 .sort_values("QUANTITY", ascending=False)
 .iloc[:10])

In [0]:
top10_value

In [0]:
top10_quant

In [0]:
# Use multiple aggregation to create both in a single table an option
# this here is just to use to compare to chart

(transactions
 .groupby("household_key")
 .agg({'SALES_VALUE': 'sum','QUANTITY': 'sum'})
 .sort_values("SALES_VALUE", ascending=False)
 .loc[:, "SALES_VALUE"]
 .describe()
)

In [0]:
# top 10 households by sales value plotted with a bar plot

top10_value["SALES_VALUE"].plot.bar()

## Product Analysis

* Which products had the most sales by sales_value? Plot  a horizontal bar chart.
* Did the top selling items have a higher than average discount rate?
* Look up the names of the  top 10 products by sales in the `products.csv` dataset.
* What was the name most common `PRODUCT_ID` among rows with the households in our top 10 households by sales value?
* Look up the product name of the item that had the highest quantity sold in a single row.

In [0]:
# Create top 10 products by sales df
# group by PRODUCT_ID and sum sales value by product
# Sort in descending order and grab top 10 rows

top10_products = (transactions
                 .groupby(["PRODUCT_ID"])
                 .agg({"SALES_VALUE": "sum"})
                 .sort_values("SALES_VALUE", ascending=False)
                 .iloc[:10]
)

In [0]:
# plot top 10 products by sale value

top10_products["SALES_VALUE"].sort_values().plot.barh()

In [0]:
# Calculate the total discount for top 10 products
# Divide that by sales value for top 10 products

((transactions
 .query("PRODUCT_ID in @top10_products.index")
 .loc[: ,"total_discount"]
 .sum()) 
/(transactions
 .query("PRODUCT_ID in @top10_products.index")
 .loc[: ,"SALES_VALUE"]
  .sum())
)

In [0]:
# read in products data

products = pd.read_csv("../project_data/product.csv")

products.head()

In [0]:
# Look up top 10 products for households in top10_value table
# Use query to reference index of top10_value to filter to relevant households
# Use value counts to get counts by product_id (this will be order in descending order)
# Then grab the top 10 products with iloc and extract the index to get product numbers

top_hh_products = (transactions
                   .query("household_key in @top10_value.index")
                   .loc[:, "PRODUCT_ID"]
                   .value_counts()
                   .iloc[:10]
                   .index
                  )

top_hh_products

In [0]:
# Filter product table to products from prior cell
products.query("PRODUCT_ID in @top_hh_products")

In [0]:
# Product with highest quantity in a single row

products.query("PRODUCT_ID == 6534178")

In [0]:
# Look up 10 product names for all customers (from first cell)

products.query("PRODUCT_ID in @top10_products.index")